In [1]:
import numpy as np
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [2]:
%matplotlib qt

In [5]:
#same as in other code
#don't need this. Build something to pull wellmap in
def init_credentials():
    global scope,path,credentials,gc,key
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    
    path = '/Users/MichaelRubsamen/Downloads/Jupyter_Notebooks/Research/JSON/hendricks-lab-jupyter-sheets-5363dda1a7e0.json'
    path = '../Credentials/hendricks-lab-jupyter-sheets-5363dda1a7e0.json'
    credentials = ServiceAccountCredentials.from_json_keyfile_name(path, scope) # Your json file here
    
    gc = gspread.authorize(credentials)
    return scope,path,credentials,gc

def pre_rxn_questions():   
    global reaction_sheet_name,key
    reaction_sheet_name = input('Enter Sheet Name as it Appears on the Spreadsheets Title: ')
    init_credentials()
    try:
        wks = gc.open(reaction_sheet_name)
        print('Everythings Ready To Go')
    except: 
        raise Exception('Spreadsheet Not Found: Make sure the spreadsheet name is spelled correctly and that it is shared with the robot ')
    
    
pre_rxn_questions() 
    
def initialize(file,*args):
    global df
    try:
        for i in args:
            sheet = i
    except: 
        sheet = input('Input Sheet Name: ')
    wks = gc.open(file)
    wks = wks.worksheet(sheet)
    data = wks.get_all_values()
    df = pd.DataFrame(data)
    header = df.iloc[0]
    df = df[1:]
    rxn_titles = header[3:].tolist()
    for i in range(len(rxn_titles)):
        rxn_titles[i] = reaction_sheet_name + ': ' + rxn_titles[i]
    df.columns = header
    return df 
initialize(reaction_sheet_name,'MSR_029_210623_Well_Map')
#MSR_029_210623

Enter Sheet Name as it Appears on the Spreadsheets Title: MSR_029_210623
Everythings Ready To Go


,Spectrometer Well,Rxn Title
1,A01,MSR_029_210623: Blank
2,B01,MSR_029_210623: 6.66 mM Cit_1.1
3,C01,MSR_029_210623: 6.66 mM Cit_1.2
4,D01,MSR_029_210623: 6.66 mM Cit_1.3
5,E01,MSR_029_210623: 6.66 mM Cit_1.4
6,F01,MSR_029_210623: 6.66 mM Cit_2.1
7,G01,MSR_029_210623: 6.66 mM Cit_2.2
8,H01,MSR_029_210623: 6.66 mM Cit_2.3
9,A02,MSR_029_210623: 6.66 mM Cit_2.4
10,B02,MSR_029_210623: 6.66 mM Cit_3.1


In [6]:

well_key = {}
def update_dict(df,well_key):
    well_key.update(dict(zip(df['Spectrometer Well'], df['Rxn Title'])))
    return well_key

In [7]:
#initialize(reaction_sheet_name,'pchem_week_Well_Map')
update_dict(df,well_key)
df_reorder = list(well_key.keys())
df_reorder

['A01',
 'B01',
 'C01',
 'D01',
 'E01',
 'F01',
 'G01',
 'H01',
 'A02',
 'B02',
 'C02',
 'D02',
 'E02',
 'F02',
 'G02',
 'H02',
 'A03']

In [8]:
datafilename = '/Volumes/GoogleDrive/Shared drives/Hendricks Lab Drive/Opentrons_Reactions/Plate Reader Data/' + input("What is your CSV name?") + '.csv'
datafilename = 'MSR_029_210623.csv'

# Read data ignoring first 50 lines
#TODO inconsistent skip rows. use a seek
df = pd.read_csv(datafilename,skiprows=50,header=None,na_values=["       -"],encoding = 'latin1',engine='python')
# Transpose the data making each plate a column
df = df.transpose()
# Fix the index column
df = df.reset_index()
# Fix the column headers, make plate names column names
headers = df.iloc[0,:]
# Drop the detector label row
df.columns = headers
headers = headers[1:]
headers = [x[:-1] for x in headers]
df = df.drop([0])
df = df.drop(columns=[0])
df = df.dropna()
#df = df.iloc[0:202589,:]
df = df.astype(float)
#df = df.iloc[:,5:]
df = df.sub(df.iloc[:,0],axis = 0) #TODO incorporate this 
df.columns = headers
print(headers)
df.tail()

What is your CSV name?
['A01', 'A02', 'A03', 'B01', 'B02', 'B03', 'C01', 'C02', 'D01', 'D02', 'E01', 'E02', 'F01', 'F02', 'G01', 'G02', 'H01', 'H02']


,A01,A02,A03,B01,B02,B03,C01,C02,D01,D02,E01,E02,F01,F02,G01,G02,H01,H02
17521,0.0,0.071,0.026,0.061,0.001,-0.072,0.128,0.008,0.065,0.004,0.064,0.010,0.078,0.035,0.070,0.034,0.064,0.033
17522,0.0,0.074,0.026,0.062,0.004,-0.069,0.129,0.010,0.065,0.006,0.065,0.011,0.080,0.037,0.072,0.037,0.065,0.035
17523,0.0,0.074,0.027,0.063,0.003,-0.070,0.128,0.010,0.067,0.006,0.065,0.011,0.080,0.038,0.073,0.037,0.066,0.035
17524,0.0,0.073,0.027,0.062,0.003,-0.070,0.128,0.010,0.065,0.007,0.064,0.010,0.078,0.035,0.072,0.035,0.065,0.034
17525,0.0,0.072,0.025,0.062,0.003,-0.068,0.126,0.010,0.064,0.006,0.064,0.010,0.079,0.035,0.070,0.035,0.067,0.034


In [9]:
print(df.columns)
print(df_reorder)
df = df[df_reorder]
headers = df.columns
#df reorder is necessary because we want the wells in order different from the order supplied by the platereader

Index(['A01', 'A02', 'A03', 'B01', 'B02', 'B03', 'C01', 'C02', 'D01', 'D02',
       'E01', 'E02', 'F01', 'F02', 'G01', 'G02', 'H01', 'H02'],
      dtype='object')
['A01', 'B01', 'C01', 'D01', 'E01', 'F01', 'G01', 'H01', 'A02', 'B02', 'C02', 'D02', 'E02', 'F02', 'G02', 'H02', 'A03']


In [10]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def plot_setup_overlay(name):
    #formats the figure nicely
    plt.figure(num=None, figsize=(4, 4),dpi=300, facecolor='w', edgecolor='k')
    plt.legend(loc="upper right",frameon = False, prop={"size":7},labelspacing = 0.5)
    plt.rc('axes', linewidth = 2)
    plt.xlabel('Wavelength (nm)',fontsize = 16, fontfamily = 'Arial',fontname="Arial")
    plt.ylabel('Absorbance (a.u.)', fontsize = 16,fontname="Arial")
    plt.tick_params(axis = "both", width = 2)
    plt.tick_params(axis = "both", width = 2)
    plt.xticks([300,400,500,600,700,800,900,1000])
    plt.yticks([i/10 for i in range(0,11,1)])
    #plt.ylims = ([0,1.6])
    plt.axis([300, 1000, 0.0 , 1.0])
    plt.xticks(fontsize = 10)
    plt.yticks(fontsize = 10)
    #plt.title(str(name) + reaction_sheet_name, fontsize = 16, pad = 20,fontname="Arial")
    plt.title(str(name), fontsize = 16, pad = 20,fontname="Arial")
    #return x
    


# def overlay(df):
#     #delete
#     y = []
#     df = df[df_reorder]
#     headers = df.columns
#     #legend_colors = []
#     for j in range(df.shape[1]):
#         temp = df.iloc[:,j].tolist()
#         y.append(temp[-701:])
#     plot_setup_overlay('Overlay: ')
#     print(len(y))
#     colors = list(cm.rainbow(np.linspace(0, 1,len(y))))
#     print(type(colors),len(colors),len(y),colors[0])
#     for i in range(len(y)):
#         plt.plot(x_vals,y[i],color = tuple(colors[i]))
#     plt.figure()
#     patches = [mpatches.Patch(color=color, label=label) for label, color in zip(headers, colors)]
#     plt.legend(patches, headers, loc='center', frameon=False)
#     legend = pd.DataFrame({'Color':patches,
#                            'Labels': headers})
   # print(legend,'LEGEND')
    
def LAM_overlay1(df):
    x_vals = list(range(300,1001))
    #overlays everything in dataframe
    y = []
    #df = df[df_reorder]
    headers = [well_key[k] for k in df.columns]
    #legend_colors = []
    for j in range(df.shape[1]):
        temp = df.iloc[:,j].tolist() #one well
        y.append(temp[-701:]) #append the last scan because that's the one you want when overlaying a lot
    plot_setup_overlay('Overlay: ')
    print(len(y))
    colors = list(cm.rainbow(np.linspace(0, 1,len(y)))) # range colors
    print(type(colors),len(colors),len(y),colors[0])
    for i in range(len(y)):
        #y is a list of lists of size 701 where each inner list is a well full spectra
        plt.plot(x_vals,y[i],color = tuple(colors[i]))
#     plt.figure() #finicky matplotlib magic (not to be confused with jupyter magic. REAL magic)
    patches = [mpatches.Patch(color=color, label=label) for label, color in zip(headers, colors)]
    plt.legend(patches, headers, loc='upper right', frameon=False,prop={'size':3}) #nice legend patches
    legend = pd.DataFrame({'Color':patches,
                           'Labels': headers})
   # print(legend,'LEGEND')
    plt.show()
    
def LAM_overlay(df,headers,title):
    x_vals = list(range(300,1001))
    #overlays only things you specify
    #TODO update so that combined 
    y = []
    #df = df[df_reorder]
    #headers = [well_key[k] for k in df.columns]
    #legend_colors = []
    for j in range(df.shape[1]):
        temp = df.iloc[:,j].tolist()
        y.append(temp[-701:])
    plot_setup_overlay(title)
    print(len(y))
    colors = list(cm.rainbow(np.linspace(0, 1,len(y))))
    print(type(colors),len(colors),len(y),colors[0])
    for i in range(len(y)):
        plt.plot(x_vals,y[i],color = tuple(colors[i]))
    patches = [mpatches.Patch(color=color, label=label) for label, color in zip(headers, colors)]
    plt.legend(patches, headers, loc='upper right', frameon=False,prop={'size':8})
    legend = pd.DataFrame({'Color':patches,
                           'Labels': headers})
    plt.figure()




LAM_overlay1(df)

No handles with labels found to put in legend.


17
<class 'list'> 17 17 [0.5 0.  1.  1. ]


findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.


In [11]:
def LAM_overlay(df,rang,title):
    #'same' as above LAM_overlay
    x_vals = list(range(300,1001))
    y = []
    #df = df[df_reorder]
    headers = [well_key[k] for k in rang]
    #legend_colors = []
    for j in rang:
        temp = df[j].tolist()
        y.append(temp[-701:])
    plot_setup_overlay(title)
    print(len(y))
    colors = list(cm.rainbow(np.linspace(0, 1,len(y))))
    print(type(colors),len(colors),len(y),colors[0])
    for i in range(len(y)):
        plt.plot(x_vals,y[i],color = tuple(colors[i]))
    patches = [mpatches.Patch(color=color, label=label) for label, color in zip(headers, colors)]
    plt.legend(patches, headers, loc='upper right', frameon=False,prop={'size':6})
    legend = pd.DataFrame({'Color':patches,
                           'Labels': headers})
LAM_overlay(df,['B01','C01','D01','E01','F01','G01'],'Aged AA/Cit Mixtures')

No handles with labels found to put in legend.


6
<class 'list'> 6 6 [0.5 0.  1.  1. ]


In [12]:
def calc_cycles(df):
    global cycles
    cycles = int(df.shape[0]/701)

In [13]:
import matplotlib.pyplot as plt

global x_val,dummy
x_vals,dummy = [],[0]
i = 300
while i <= 1000:
    x_vals.append(i)
    i += 1
dummy = dummy * 701

def ind_subplots(df):
    df = df.tail(701)
    #in an ideal world this plots in shape of platereader
    fig, axes = plt.subplots(8, 10, figsize=(50, 50),subplot_kw=dict(box_aspect=1))
                              #,sharex = True,sharey = True)
    for idx, (col, ax) in enumerate(zip(df.columns, axes.flatten())):
        try:
            ax.set_title(col + ": " + well_key[col])
            ax.set_title(col)
            ax.plot(x_vals, df[col],color = 'red')
        except:
            ax.plot(x_vals,dummy,color = 'red')
            ax.set_title('NOT DATA')
        #plt.title(str(name) + well_key[name])
        plt.subplots_adjust(wspace=0.3, hspace= -0.1)
    
        ax.tick_params(
            which='both',
            bottom='off',
            left='off',
            right='off',
            top='off'
        )
        ax.set_xlim((300,1000))
        ax.set_ylim((0,1.0))
        ax.set_xlabel("Wavlength (nm)")
        ax.set_ylabel("Absorbance (A.U.)")
        ax.set_xticks(range(300, 1100, 100))
        #ax.set_aspect(adjustable='box')
        #ax.set_yticks(range(0,1))
    else:
        [ax.set_visible(False) for ax in axes.flatten()[idx+1:]]
#don't want reordered here. Want to 
ind_subplots(df)

In [15]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#plots kinetics for all of the wells

def kinetics_subplots(df):
    calc_cycles(df)
    colors = list(cm.rainbow(np.linspace(0, 1, cycles)))
    fig, axes = plt.subplots(8, 12, dpi=300, figsize=(50, 50),subplot_kw=dict(box_aspect=1,sharex = True,sharey = True))
    for idx, (col, ax) in enumerate(zip(df.columns, axes.flatten())):
        try:
            ax.set_title(well_key[col])
            kin = 0
            temp = df[col].tolist()
            while kin != cycles:
                ax.plot(x_vals, temp[:701], color = tuple(colors[kin]))
                del temp[:701]
                kin +=1
        except:
            ax.plot(x_vals,dummy,color = 'red')
            ax.set_title('NOT DATA')
        plt.subplots_adjust(wspace=0.3, hspace= -0.1)
    
        ax.tick_params(
            which='both',
            bottom='off',
            left='off',
            right='off',
            top='off'
        )
        ax.set_xlim((300,1000))
        ax.set_ylim((0,1.0))
        ax.set_xlabel("Wavlength (nm)")
        ax.set_ylabel("Absorbance (A.U.)")
        ax.set_xticks(range(300, 1100, 100))
        #ax.set_aspect(adjustable='box')
        #ax.set_yticks(range(0,1))
    else:
        [ax.set_visible(False) for ax in axes.flatten()[idx+1:]]
kinetics_subplots(df)
def ind_kinetics(df,idx):
    calc_cycles(df)
    colors = list(cm.rainbow(np.linspace(0, 1, cycles)))
    kin = 0
    temp = df.iloc[:,idx].tolist()
    plot_setup_overlay('Overlay: ')
    while kin != cycles:
        plt.plot(x_vals, temp[:701], color = tuple(colors[kin]))
        del temp[:701]
        kin +=1


In [ ]:
def ind_kinetics(df,idx,lamb,step):
    calc_cycles(df)
    temp = df.iloc[:,idx]
    temp = temp[lamb:-1:701]
    x_vals = [0]
    time = 0
    while len(x_vals) != len(temp):
        x_vals.append(time+step)
        time += step
    print(temp,len(x_vals))
    plt.plot(x_vals,temp)
    plt.show()
ind_kinetics(df,6,110,0.5) 



In [ ]:
len(df.iloc[:,2])/701

In [ ]:
lambda max list
abs at max list
time list

In [ ]:
test time things sitting around
test age of ascorbic acid
big plate of checking a bunch of different paramters.